In [ ]:
import pandas as pd
import numpy as np
from seaborn import heatmap

In [ ]:
df=pd.read_csv("/Users/keith/Documents/projects/Learning/Udacity/Data Science Nanodegree/data/StackOverflow Survey/developer_survey_2017/survey_results_public.csv")


In [ ]:
datafrom=['Salary','JobSatisfaction', 'Country', 'HomeRemote']
df = df[datafrom].copy()
df.dropna(axis=0, how='any', inplace=True)


In [ ]:
def computeMutualInformation(tupleWithHist2d):
    hist2d = tupleWithHist2d[0]
    total = hist2d.sum(axis=0).sum()
    px = hist2d.sum(axis=1)/total
    py = hist2d.sum(axis=0)/total
    pbar = np.outer(px, py)
    p = hist2d/total
    lgp = np.log2(p) - np.log2(pbar)
    return (p*lgp)[np.isfinite(lgp)].sum()


In [ ]:
def mutualInfoMatrix(df):
    columns = df.columns
    n = len(columns)
    
    mtrx = np.zeros((n, n), dtype=float)
    
    for i in range(n):
        for j in range(n):
            # I know, I know.
            mtrx[i, j] = computeMutualInformation(binPair(df[[columns[i], columns[j]]]))
    
    return mtrx


###### Binning Wrapper Function That Handles Both Quant & Cat Features

In [ ]:
def binPair(df):
    # TODO: require df to have precisely two columns    
    
    indices = [pd.Index([]), pd.Index([])]
    
    for j in range(2):
        srs = df.iloc[:, j]
        indices[j] = srs.value_counts(bins=10, sort=False).index if srs.dtype=='float' else srs.value_counts().index
    
    ns = [0, 0]
    ns[0] = len(indices[0])
    ns[1] = len(indices[1])
    
    hist2d = np.zeros(shape=(len(indices[0]), len(indices[1])), dtype=int)
    for _, row in df.iterrows():
        idxs = [0, 0]
        idxs[0] = indices[0].get_loc(row.iloc[0])
        idxs[1] = indices[1].get_loc(row.iloc[1])
        if idxs[0] not in range(ns[0]) or idxs[1] not in range(ns[1]):
            continue
        hist2d[idxs[0], idxs[1]] += 1

    return hist2d, indices[0], indices[1]
        
        

In [ ]:
matrix = mutualInfoMatrix(df)

In [ ]:
bx = heatmap(matrix, annot=True, xticklabels=df.columns, yticklabels=df.columns)